In [1]:
import tensorflow as tf
import tflearn
from tflearn.layers.conv import conv_2d,max_pool_2d
from tflearn.layers.core import input_data,dropout,fully_connected
from tflearn.layers.estimator import regression
import numpy as np
import cv2
from sklearn.utils import shuffle

/Users/admin/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/admin/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/admin/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/admin/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dty

In [2]:
#Load Images for Open Eyes
loadedImages = []
for i in range(0, 100):
    image = cv2.imread('Dataset/OpenEyesImages/Train/open_' + str(i) + '.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    loadedImages.append(gray_image.reshape(89, 100, 1))

#Load Images for Closed Eyes
for i in range(0, 100):
    image = cv2.imread('Dataset/ClosedEyesImages/Train/closed_' + str(i) + '.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    loadedImages.append(gray_image.reshape(89, 100, 1))
    
#Load Images From Fist
# for i in range(0, 1000):
#     image = cv2.imread('Dataset/FistImages/fist_' + str(i) + '.png')
#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     loadedImages.append(gray_image.reshape(89, 100, 1))


In [3]:
# Create OutputVector

outputVectors = []
for i in range(0, 100):
    outputVectors.append([1, 0, 0])

for i in range(0, 100):
    outputVectors.append([0, 1, 0])

# for i in range(0, 3):
#     outputVectors.append([0, 0, 1])


In [4]:
testImages = []

#Load Images for Open Eyes
for i in range(0, 100):
    image = cv2.imread('Dataset/OpenEyesImages/Test/open_' + str(i) + '.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    testImages.append(gray_image.reshape(89, 100, 1))

#Load Images for Closed Eyes
for i in range(0, 100):
    image = cv2.imread('Dataset/ClosedEyesImages/Test/closed_' + str(i) + '.png')
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    testImages.append(gray_image.reshape(89, 100, 1))
    
#Load Images for Fist
# for i in range(0, 100):
#     image = cv2.imread('Dataset/FistTest/fist_' + str(i) + '.png')
#     gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     testImages.append(gray_image.reshape(89, 100, 1))

testLabels = []

for i in range(0, 100):
    testLabels.append([1, 0, 0])
    
for i in range(0, 100):
    testLabels.append([0, 1, 0])

# for i in range(0, 3):
#     testLabels.append([0, 0, 1])

In [5]:
# Define the CNN Model
tf.reset_default_graph()
convnet=input_data(shape=[None,89,100,1],name='input')
convnet=conv_2d(convnet,32,2,activation='relu')
convnet=max_pool_2d(convnet,2)
convnet=conv_2d(convnet,64,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,128,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,256,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,256,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,128,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=conv_2d(convnet,64,2,activation='relu')
convnet=max_pool_2d(convnet,2)

convnet=fully_connected(convnet,1000,activation='relu')
convnet=dropout(convnet,0.75)

convnet=fully_connected(convnet,3,activation='softmax')

convnet=regression(convnet,optimizer='adam',learning_rate=0.001,loss='categorical_crossentropy',name='regression')

model=tflearn.DNN(convnet,tensorboard_verbose=0)



Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Instructions for updating:
keep_dims is deprecated, use keepdims instead

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [6]:
# Shuffle Training Data
loadedImages, outputVectors = shuffle(loadedImages, outputVectors, random_state=0)

# Train model
model.fit(loadedImages, outputVectors, n_epoch=50,
           validation_set = (testImages, testLabels),
           snapshot_step=100, show_metric=True, run_id='convnet_coursera')

model.save("TrainedModel/EyesRecogModel.tfl")

Training Step: 199  | total loss: 0.51150 | time: 1.039s
| Adam | epoch: 050 | loss: 0.51150 - acc: 0.8893 -- iter: 192/200
Training Step: 200  | total loss: 0.48720 | time: 2.109s
| Adam | epoch: 050 | loss: 0.48720 - acc: 0.8878 | val_loss: 0.15265 - val_acc: 0.9850 -- iter: 200/200
--
INFO:tensorflow:/Users/admin/Repo/Hand-Gesture-Recognition-Using-Background-Elllimination-and-Convolution-Neural-Network-Akhil/TrainedModel/EyesRecogModel.tfl is not in all_model_checkpoint_paths. Manually adding it.
